---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [4]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [5]:
import re
def ans_one():
    #df = pd.read_csv('university_towns.txt')
    ut_list = []
    i = 0
    with open('university_towns.txt', 'r') as f:
        for line in f:
            i += 1
            if re.search('\[edit]', line):
                state = re.sub('\[edit]','',line).strip()
                i -= 1
            elif re.search('\(.*', line):
                region = re.sub('\(.*', '', line).strip()
                ut_list.append([state, region])
            else:
                region = line.strip()
                ut_list.append([state, region])
    f.close()
    df = pd.DataFrame(ut_list, columns = ['State', 'RegionName'])
    return df

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    return ans_one()
ans_one()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [198]:
def ans_two():
    gdp = pd.read_excel('gdplev.xls')
    gdp.drop(gdp.columns[[0,1,2,3,5,7]], axis = 1, inplace = True)
    gdp.columns = ['YearQuarters', 'GDP(billions of chained 2009 dollars)']
    gdp = gdp[219:]
    gdp.reset_index(inplace = True)
    gdp.drop(gdp.columns[[0]], axis = 1, inplace = True)
    
    for i in range(4, len(gdp)):
        if (gdp.iloc[i-4, 1] > gdp.iloc[i-3, 1]) and (gdp.iloc[i-3, 1] > gdp.iloc[i-2, 1]) and (gdp.iloc[i-2, 1] < gdp.iloc[i-1, 1]) and (gdp.iloc[i-1, 1] < gdp.iloc[i, 1]):
             recession_base_idx = i-4
    
    result = gdp.iloc[recession_base_idx, 0]
    j = recession_base_idx
    while True:
        if (gdp.iloc[j-1, 1] > gdp.iloc[j, 1]):
            j -= 1
        else:
            result = gdp.iloc[j+1, 0]
            break
        
    return result

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return ans_two()
ans_two()

'2008q3'

In [199]:
def ans_three():
    gdp = pd.read_excel('gdplev.xls')
    gdp.drop(gdp.columns[[0,1,2,3,5,7]], axis = 1, inplace = True)
    gdp.columns = ['YearQuarters', 'GDP(billions of chained 2009 dollars)']
    gdp = gdp[219:]
    gdp.reset_index(inplace = True)
    gdp.drop(gdp.columns[[0]], axis = 1, inplace = True)
    
    for i in range(4, len(gdp)):
        if (gdp.iloc[i-4, 1] > gdp.iloc[i-3, 1]) and (gdp.iloc[i-3, 1] > gdp.iloc[i-2, 1]) and (gdp.iloc[i-2, 1] < gdp.iloc[i-1, 1]) and (gdp.iloc[i-1, 1] < gdp.iloc[i, 1]):
             return gdp.iloc[i,0]
            
            
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    return ans_three()
ans_three()

'2009q4'

In [200]:
def ans_four():
    gdp = pd.read_excel('gdplev.xls')
    gdp.drop(gdp.columns[[0,1,2,3,5,7]], axis = 1, inplace = True)
    gdp.columns = ['YearQuarters', 'GDP(billions of chained 2009 dollars)']
    gdp = gdp[219:]
    gdp.reset_index(inplace = True)
    gdp.drop(gdp.columns[[0]], axis = 1, inplace = True)
    
    for i in range(4, len(gdp)):
        if (gdp.iloc[i-4, 1] > gdp.iloc[i-3, 1]) and (gdp.iloc[i-3, 1] > gdp.iloc[i-2, 1]) and (gdp.iloc[i-2, 1] < gdp.iloc[i-1, 1]) and (gdp.iloc[i-1, 1] < gdp.iloc[i, 1]):
             return gdp.iloc[i-2,0]
        
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    return ans_four()
ans_four()

'2009q2'

In [213]:
import pandas as pd
import numpy as np

states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

def answer_five():
    hous = pd.read_csv('City_Zhvi_AllHomes.csv')
    #hous = hous.fillna(0)
    req_cols = hous.loc[:, hous.columns.str.startswith('20')]
    
    req_prev = req_cols.columns
    req_cols_names = req_cols.columns.tolist()
    q = ['q1','q2','q3','q4']
    
    q_index = 0
    i = 0

    while i+3 < (len(req_cols_names)):
        new_col = req_cols_names[i][0:4] + q[(q_index)%4]
        roo = [req_cols_names[i], req_cols_names[i+1], req_cols_names[i+2], req_cols_names[i+3]]
        req_cols[new_col] = req_cols.apply(lambda row : np.mean(row[roo]), axis = 1)
        i += 3 
        q_index += 1

    req_cols['2016q3'] = req_cols.apply(lambda row : np.mean(row[['2016-07', '2016-08']]), axis = 1)
    req_cols.drop(req_prev, axis = 1, inplace = True)
    
    hous = hous[['RegionName', 'State']]
    abbr = pd.DataFrame(pd.Series(states))
    
    hous = pd.merge(hous, abbr, how = "inner", left_on = 'State', right_index = True)
    
    hous.columns = ['RegionName', 'StateAbr', 'State']
    hous.drop(['StateAbr'], axis = 1, inplace = True)
    
    hous = pd.merge(hous, req_cols, how = "inner", left_index = True, right_index = True)
    
    hous.set_index(['State', 'RegionName'], inplace = True)
    
    return hous

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return answer_five()

In [214]:
from scipy.stats import ttest_ind

def ans_final():
    univ_towns = get_list_of_university_towns()
    univ_towns['univ_town'] = True
    univ_towns.set_index(['State', 'RegionName'], inplace = True)
    
    hous_qua = convert_housing_data_to_quarters()
    
    r_start = get_recession_start()
    r_bottom =  get_recession_bottom()
    start = hous_qua.columns[hous_qua.columns.get_loc(r_start)-1]
    
    hous_qua['ratio'] = hous_qua[start] - hous_qua[r_bottom]
    hous_qua = hous_qua[[start, r_bottom, 'ratio']]
    
    
    df_tmp = pd.merge(hous_qua, univ_towns, how = "inner", left_index = True, right_index = True).reset_index()
    hous_qua.reset_index(inplace = True)
    df = pd.merge(hous_qua, df_tmp, how = "outer",  on = ['State', 'RegionName', start, r_bottom, 'ratio'])
    df['univ_town'] = df['univ_town'].fillna(False)
    
    ut = df[df['univ_town'] == True]
    nut = df[df['univ_town'] == False]
    
    t,p = ttest_ind(ut['ratio'].dropna(), nut['ratio'].dropna())
    
    different = True if p < 0.01 else False
    
    better = "university town" if ut['ratio'].mean() < nut['ratio'].mean() else "non-university town"
    
    return (different, p, better)

def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return ans_final()
ans_final()

<ipython-input-213-674599734941>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_cols[new_col] = req_cols.apply(lambda row : np.mean(row[roo]), axis = 1)


(True, 0.0018000091375413134, 'university town')